In [47]:
import pandas as pd

# Load the original CSV file
df = pd.read_csv("/home/rohank__iitp/Work/niladri/phi/conversation.csv")  # replace with actual path if not in current directory

# Prepare variables
output_rows = []
agent_responses = []

# Temporary variables to track user query and agent reply
current_user_query = None
current_conversation_id = None
record_id = 1

for index, row in df.iterrows():
    if row['speaker'] == 'User':
        agent_responses.append(row['id'])
        if current_user_query is not None:
            # Save previous user query without agent reply
            output_rows.append({
                'id': record_id,
                'conversation_id': current_conversation_id,
                'user_query': current_user_query,
                'agent_reply': ''
            })
            record_id += 1
        # Start new user query
        current_user_query = row['utterance']
        current_conversation_id = row['conversation_id']
    elif row['speaker'] == 'Agent':
        
        if current_user_query is not None:
            
            # Save user query and agent reply
            output_rows.append({
                'id': record_id,
                'conversation_id': current_conversation_id,
                'user_query': current_user_query,
                'agent_reply': row['utterance']
            })
            record_id += 1
            current_user_query = None  # Reset after pairing

# If last entry is a user query without an agent reply
if current_user_query is not None:
    output_rows.append({
        'id': record_id,
        'conversation_id': current_conversation_id,
        'user_query': current_user_query,
        'agent_reply': ''
    })

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv("/home/rohank__iitp/Work/niladri/phi/router/table_router.csv", index=False)

# Print or return list of agent responses
print(agent_responses)


[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243]


In [48]:
len(agent_responses)

122

In [49]:
import pandas as pd
import json

df=pd.read_csv("/home/rohank__iitp/Work/niladri/phi/router/table_router.csv")  # replace with actual path if not in current directory
# Output list for matching answers
matching_answers = []

# Read and process the JSONL file
with open('/home/rohank__iitp/Work/niladri/phi/router/cleaned_output.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        # if data.get('id_json') in agent_responses:
        matching_answers.append(data.get('answer'))

df["router agent response"]= matching_answers
df.to_csv("/home/rohank__iitp/Work/niladri/phi/router/table_router.csv", index=False)

In [6]:
import sys
sys.path.append('/DATA/rohan_kirti/.local/lib/python3.8/site-packages')

In [50]:
import pandas as pd
from rouge_score import rouge_scorer

# Load CSV
df = pd.read_csv("/home/rohank__iitp/Work/niladri/phi/router/table_router.csv")  # Replace with your actual file path

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

# Compute ROUGE scores
def compute_rouge(row):
    scores = scorer.score(row['router agent response'], row['agent_reply'])
    return scores['rouge1'].fmeasure  # You can also use .precision or .recall

# Apply ROUGE calculation
df['rouge'] = df.apply(compute_rouge, axis=1)

# Save the updated DataFrame
df.to_csv("/home/rohank__iitp/Work/niladri/phi/router/table_router.csv", index=False)